<a href="https://colab.research.google.com/github/kwanda2426/load-shortfall-regression-predict-api-template/blob/master/latest_note_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression Predict Student Solution

© Explore Data Science Academy

In [2]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from statsmodels.graphics.correlation import plot_corr
from scipy.stats import skew
import statistics
#ignoring warnings
import warnings
warnings.simplefilter(action='ignore')
import datetime

# Libraries for data preparation and model building
import statsmodels.formula.api as sm
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
# saving my model
import pickle

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Loading the Data

In [3]:
#making sure that we can see all rows and cols
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#reading the data from the csv

df_train = pd.read_csv('https://raw.githubusercontent.com/kwanda2426/load-shortfall-regression-predict-api-template/master/Notebook/df_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/kwanda2426/load-shortfall-regression-predict-api-template/master/Notebook/df_test.csv')
test_df = df_test.copy()
train_df = df_train.copy()



# Preprocessing

It is a good idea to concatenate the train and test data to ensure that any changes are consistent across the train and test data.

In [ ]:
# Combining the train and test data sets

df_concat = pd.concat([train_df, test_df])


In [ ]:
# Checking the shape of the new data frame

df_concat.shape

(11683, 49)

In [ ]:
#Show the first five rows of the dataset

df_concat.head()

,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,Seville_clouds_all,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,Madrid_clouds_all,Seville_wind_speed,Barcelona_rain_1h,Seville_pressure,Seville_rain_1h,Bilbao_snow_3h,Barcelona_pressure,Seville_rain_3h,Madrid_rain_1h,Barcelona_rain_3h,Valencia_snow_3h,Madrid_weather_id,Barcelona_weather_id,Bilbao_pressure,Seville_weather_id,Valencia_pressure,Seville_temp_max,Madrid_pressure,Valencia_temp_max,Valencia_temp,Bilbao_weather_id,Seville_temp,Valencia_humidity,Valencia_temp_min,Barcelona_temp_max,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h
0,0,2015-01-01 03:00:00,0.666667,level_5,0.0,0.666667,74.333333,64.000000,0.0,1.000000,0.0,223.333333,6.333333,42.666667,0.0,3.333333,0.0,sp25,0.0,0.0,1036.333333,0.0,0.0,0.0,0.0,800.0,800.0,1035.000000,800.0,1002.666667,274.254667,971.333333,269.888000,269.888000,800.0,274.254667,75.666667,269.888000,281.013000,265.938000,281.013000,269.338615,269.338615,281.013000,269.338615,274.254667,265.938000,265.938000,6715.666667
1,1,2015-01-01 06:00:00,0.333333,level_10,0.0,1.666667,78.333333,64.666667,0.0,1.000000,0.0,221.000000,4.000000,139.000000,0.0,3.333333,0.0,sp25,0.0,0.0,1037.333333,0.0,0.0,0.0,0.0,800.0,800.0,1035.666667,800.0,1004.333333,274.945000,972.666667,271.728333,271.728333,800.0,274.945000,71.000000,271.728333,280.561667,266.386667,280.561667,270.376000,270.376000,280.561667,270.376000,274.945000,266.386667,266.386667,4171.666667
2,2,2015-01-01 09:00:00,1.000000,level_9,0.0,1.000000,71.333333,64.333333,0.0,1.000000,0.0,214.333333,2.000000,326.000000,0.0,2.666667,0.0,sp25,0.0,0.0,1038.000000,0.0,0.0,0.0,0.0,800.0,800.0,1036.000000,800.0,1005.333333,278.792000,974.000000,278.008667,278.008667,800.0,278.792000,65.666667,278.008667,281.583667,272.708667,281.583667,275.027229,275.027229,281.583667,275.027229,278.792000,272.708667,272.708667,4274.666667
3,3,2015-01-01 12:00:00,1.000000,level_8,0.0,1.000000,65.333333,56.333333,0.0,1.000000,0.0,199.666667,2.333333,273.000000,0.0,4.000000,0.0,sp25,0.0,0.0,1037.000000,0.0,0.0,0.0,0.0,800.0,800.0,1036.000000,800.0,1009.000000,285.394000,994.666667,284.899552,284.899552,800.0,285.394000,54.000000,284.899552,283.434104,281.895219,283.434104,281.135063,281.135063,283.434104,281.135063,285.394000,281.895219,281.895219,5075.666667
4,4,2015-01-01 15:00:00,1.000000,level_7,0.0,1.000000,59.000000,57.000000,2.0,0.333333,0.0,185.000000,4.333333,260.000000,0.0,3.000000,0.0,sp25,0.0,0.0,1035.000000,0.0,0.0,0.0,0.0,800.0,800.0,1035.333333,800.0,NaN,285.513719,1035.333333,283.015115,283.015115,800.0,285.513719,58.333333,283.015115,284.213167,280.678437,284.213167,282.252063,282.252063,284.213167,282.252063,285.513719,280.678437,280.678437,6620.666667


* #### Check for Missing Values



This is done so that we may know whether to remove the column or to impute data.

In [ ]:
# checking for the percentage of missing data

round((df_concat.isnull().sum()/df_concat.shape[0])
      *100,2).astype(str)+ ' %'

Unnamed: 0                0.0 %
time                      0.0 %
Madrid_wind_speed         0.0 %
Valencia_wind_deg         0.0 %
Bilbao_rain_1h            0.0 %
Valencia_wind_speed       0.0 %
Seville_humidity          0.0 %
Madrid_humidity           0.0 %
Bilbao_clouds_all         0.0 %
Bilbao_wind_speed         0.0 %
Seville_clouds_all        0.0 %
Bilbao_wind_deg           0.0 %
Barcelona_wind_speed      0.0 %
Barcelona_wind_deg        0.0 %
Madrid_clouds_all         0.0 %
Seville_wind_speed        0.0 %
Barcelona_rain_1h         0.0 %
Seville_pressure          0.0 %
Seville_rain_1h           0.0 %
Bilbao_snow_3h            0.0 %
Barcelona_pressure        0.0 %
Seville_rain_3h           0.0 %
Madrid_rain_1h            0.0 %
Barcelona_rain_3h         0.0 %
Valencia_snow_3h          0.0 %
Madrid_weather_id         0.0 %
Barcelona_weather_id      0.0 %
Bilbao_pressure           0.0 %
Seville_weather_id        0.0 %
Valencia_pressure       21.59 %
Seville_temp_max          0.0 %
Madrid_p

#### Key Findings:
* The feature ‘Valencia_pressure’ has 21.59% missing which can be imputed.
* ‘load_shortfall_3h’ has 24.99% missing data, but this is the part of test data that was concatenated which is not necessarily needed.

In [ ]:
# Impute missing values for Valencia_pressure with mode

df_concat['Valencia_pressure'].fillna(statistics.mode(df_concat['Valencia_pressure'])
                                   , inplace = True)

The missing values for pressure were imputed using the mode because, naturally pressure does not vary by a significant quantity. 

* #### Checking that there are no missing values on columns other than load_shortfall_3h

In [ ]:
# checking for the percentage of missing data

round((df_concat.isnull().sum()/df_concat.shape[0])
      *100,2).astype(str)+ ' %'

Unnamed: 0                0.0 %
time                      0.0 %
Madrid_wind_speed         0.0 %
Valencia_wind_deg         0.0 %
Bilbao_rain_1h            0.0 %
Valencia_wind_speed       0.0 %
Seville_humidity          0.0 %
Madrid_humidity           0.0 %
Bilbao_clouds_all         0.0 %
Bilbao_wind_speed         0.0 %
Seville_clouds_all        0.0 %
Bilbao_wind_deg           0.0 %
Barcelona_wind_speed      0.0 %
Barcelona_wind_deg        0.0 %
Madrid_clouds_all         0.0 %
Seville_wind_speed        0.0 %
Barcelona_rain_1h         0.0 %
Seville_pressure          0.0 %
Seville_rain_1h           0.0 %
Bilbao_snow_3h            0.0 %
Barcelona_pressure        0.0 %
Seville_rain_3h           0.0 %
Madrid_rain_1h            0.0 %
Barcelona_rain_3h         0.0 %
Valencia_snow_3h          0.0 %
Madrid_weather_id         0.0 %
Barcelona_weather_id      0.0 %
Bilbao_pressure           0.0 %
Seville_weather_id        0.0 %
Valencia_pressure         0.0 %
Seville_temp_max          0.0 %
Madrid_p

In [ ]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11683 entries, 0 to 2919
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            11683 non-null  int64  
 1   time                  11683 non-null  object 
 2   Madrid_wind_speed     11683 non-null  float64
 3   Valencia_wind_deg     11683 non-null  object 
 4   Bilbao_rain_1h        11683 non-null  float64
 5   Valencia_wind_speed   11683 non-null  float64
 6   Seville_humidity      11683 non-null  float64
 7   Madrid_humidity       11683 non-null  float64
 8   Bilbao_clouds_all     11683 non-null  float64
 9   Bilbao_wind_speed     11683 non-null  float64
 10  Seville_clouds_all    11683 non-null  float64
 11  Bilbao_wind_deg       11683 non-null  float64
 12  Barcelona_wind_speed  11683 non-null  float64
 13  Barcelona_wind_deg    11683 non-null  float64
 14  Madrid_clouds_all     11683 non-null  float64
 15  Seville_wind_speed  

#### Key Findings:

* Indeed only ‘load_shortfall_3h’ reflects 24.99% of missing data, which is what we expected as shown by Non-Null Count.

* #### Deleting irrelevant column(s)

They need to be removed because they have no relavancy to our data set as they are not included in the information supplied.

In [ ]:
# Dropping the 'Unnamed: 0' column

df_concat.drop(['Unnamed: 0'], inplace = True, axis = 1)

# Exploratory Data Analysis

# Feature Engineering

In [ ]:
# Keeping a clean copy of our dataframe

df_clean = df_concat.copy()

#### Engineering new features from the 'time' feature

As shown by one of the plots, time has a relationship with the load_shortfall_3h. These are separated into four, i.e year, month, day and hour features.

In [ ]:
# Changing 'time' date type from object to 'datetime64'

df_clean['time'] = pd.to_datetime(df_clean['time'])

# Creating new features

df_clean['year'] = df_clean['time'].dt.year # year feature

df_clean['month'] = df_clean['time'].dt.month # month feature

df_clean['day'] = df_clean['time'].dt.day # day

df_clean['hour'] = df_clean['time'].dt.hour #hour

#### Changing Categorical Data into Numeric data

All input data for regression model building purposes needs to be numerical.

In [ ]:
#Extracting numbers from Valencia_wind_deg 

df_clean['Valencia_wind_deg'] = df_clean['Valencia_wind_deg'].str.extract('(\d+)')

#Changing Valencia_wind_deg data type from object to numeric

df_clean['Valencia_wind_deg'] = pd.to_numeric(df_clean['Valencia_wind_deg'])

#Extracting numbers from Seville_pressure
  
df_clean['Seville_pressure'] = df_clean['Seville_pressure'].str.extract('(\d+)')

#Changing Seville_pressure data type from object to numeric

df_clean['Seville_pressure'] = pd.to_numeric(df_clean['Seville_pressure'])

# Feature Selection

Next step is to do a data scaling to allow our features to be on the same scale. As shown by the our analysis of our data using 'kurtosis', our data has some outliers. Normalisation will not be a good thing to do, because the problem with normalisation, is that if there are outliers, the bulk of our data will all lie in a small range, so we would lose information.

standardisation, on the other hand, does not suffer from this drawback as it deals outliers efficiently. Our features can now take on a range which includes negative values, centred around the mean of zero. This means that all our variables won't be bound to the exact same range.

In [ ]:
# Scaling the data 

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

X_test_Scaled = scaler.fit_transform(X_t)

We apply 'train_test_split' to split the data into train and test data. 80% is reserved for training and 20% for testing our model. This allows us to monitor the performance of our model.

In [ ]:
# Train-test split

X_train, X_test, y_train, y_test = train_test_split(X_scaled,
                                                    y,
                                                    test_size = 0.20,
                                                    random_state = 42,
                                                    shuffle = True)


# Model Training

We will train different models on the training data.

In [ ]:
# Creating the linear regression model 

l_model = LinearRegression() #create the model

l_model.fit(X_train, y_train) #train the model



LinearRegression()

In [ ]:
# Creating the Ridge model

ridge_model = Ridge() #create the model

ridge_model.fit(X_train, y_train) #train the model

Ridge()

In [ ]:
# Creating the LASSO model, setting alpha to 0.01

lasso_model = Lasso(alpha = 0.01) #create the model

lasso_model.fit(X_train,y_train) #train the model


Lasso(alpha=0.01)

In [ ]:
# Creating the Regression Tree model with maximum depth of 2

regr_tree = DecisionTreeRegressor( max_depth = 3,random_state = 42) #create the model

regr_tree.fit(X_train,y_train) #train the model



DecisionTreeRegressor(max_depth=3, random_state=42)

# Model's predictive accuracy assessment

We compare the RMSE for the train data and for test data. We also compare using a plot.

In [ ]:
# LinearRegression predictions

lr_train = l_model.predict(X_train) #predict y_train

lr_test = l_model.predict(X_test) #predict y_test

In [ ]:
# Ridge prediction

train_ridge = ridge_model.predict(X_train) #predict y_train

test_ridge = ridge_model.predict(X_test) #predict y_test


In [ ]:
# LASSO prediction

train_lasso = lasso_model.predict(X_train) #predict y_train

test_lasso = lasso_model.predict(X_test) #predict y_test


In [ ]:
# Decision Tree prediction

regr_tree_train = regr_tree.predict(X_train) #predict y_train

regr_tree_test = regr_tree.predict(X_test) #predict y_test

Let's put the results into a dictionary.

In [ ]:
# Dictionary of results
results_dict = {'Training RMSE':
                    {
                        "LinearRegression": np.sqrt(metrics.mean_squared_error(y_train, lr_train)),
                        "Ridge": metrics.mean_squared_error(y_train, train_ridge),
                        "LASSO": metrics.mean_squared_error(y_train, train_lasso),
                        "Decision Tree": metrics.mean_squared_error(y_train, regr_tree_train)
                    },
                    'Test RMSE':
                    {
                        "LinearRegression": np.sqrt(metrics.mean_squared_error(y_test, lr_test)),
                        "Ridge": metrics.mean_squared_error(y_test, test_ridge),
                        "LASSO": metrics.mean_squared_error(y_test, test_lasso),
                        "Decision Tree": metrics.mean_squared_error(y_test, regr_tree_test)
                    }
                }

In [ ]:
# Create dataframe from dictionary

results_df = pd.DataFrame(data = results_dict)

We'll plot the the test set versus four methods:

- LinearRegression
- Ridge 
- LASSO 
- Decision Tree.

In [ ]:
#plt.plot(np.arange(96,120), lasso_model.predict(X_test), label = 'LASSO')
#plt.plot(np.arange(96,120), ridge_model.predict(X_test), label = 'Ridge')
#plt.plot(np.arange(96,120), l_model.predict(X_test), label = 'LinearRegression')
#plt.plot(np.arange(96,120), regr_tree.predict(X_test), label = 'Decision Tree')
#plt.plot(np.arange(96,120), y_test, label = 'Testing')
#plt.legend()

#plt.show()

In [ ]:
Ridged = ridge_model.predict(X_test_Scaled)

regr_tre = regr_tree.predict(X_test_Scaled)

lasso_mod = lasso_model.predict(X_test_Scaled)

lr_model = l_model.predict(X_test_Scaled)

In [ ]:
dict_ = {'Ridge': Ridged, 
         'Decision Tree': regr_tre,
         'LASSO' : lasso_mod,
         'Linear Regression' :lr_model
        }

In [ ]:
Results_ = pd.DataFrame(data = dict_)

In [ ]:
pre_dict = {'time':Time,'load_shortfall_3h': lasso_mod} # dictionary containing predicted values

predict_ed = pd.DataFrame(pre_dict) # dataframe with the date and load_shortfall_3h

predict_ed.to_csv('Te_1.csv',index = False)